## First dataset and tokenization attempt

In [ ]:
with open("Harry_Potter_all_char_separated.txt", "r") as file:
    lines = []
    for line in file:
        lines = line.split("|")
    print(f"Number of lines :{len(lines)}")


In [ ]:
with open("Harry_Potter_all_char_separated.txt", "r") as file:
    chars = []
    for line in file:
        for char in line:
            if char != "|":
                chars.append(char)

    print(f"Number of characters :{len(chars)}")



Number of characters :6435489


In [ ]:
from collections import Counter

counter = Counter(chars)
print(counter)

keys = counter.keys()
tokens = sorted(list(keys)) #character level tokenization
print(f"Nuber of tokens :{len(tokens)}\n")

for t in tokens:
    print(t, end="")

Counter({' ': 1338782, 'e': 570968, 't': 392008, 'a': 375735, 'o': 366717, 'n': 309034, 'r': 303917, 'i': 296392, 'h': 280449, 's': 278706, 'd': 231033, 'l': 198557, 'u': 138432, 'g': 117841, 'y': 112596, 'w': 107532, 'm': 103770, 'c': 96127, 'f': 90860, ',': 86100, 'p': 73341, 'b': 69118, '.': 62382, 'k': 53744, 'v': 39172, 'H': 38396, '“': 37038, '”': 36415, '’': 34770, 'I': 19208, 'T': 13178, 'S': 11660, 'M': 11096, '?': 10877, 'W': 10263, 'D': 10031, 'R': 9413, 'A': 9244, 'P': 7549, '!': 6943, 'x': 6365, 'B': 6176, 'G': 6098, 'C': 5357, 'N': 4946, 'j': 4860, 'L': 4855, 'Y': 4788, 'F': 4704, 'z': 4468, 'O': 4453, 'E': 4194, 'q': 3495, 'V': 2394, 'U': 2069, 'K': 1824, ':': 1630, 'Q': 822, 'J': 804, '‘': 468, ')': 379, '(': 327, '1': 178, 'Z': 127, 'X': 127, '2': 37, '3': 29, '4': 23, '7': 21, '9': 20, '0': 17, '5': 15, '6': 13, '8': 11, '&': 1})
Nuber of tokens :75

 !&(),.0123456789:?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz‘’“”

## New Dataset of Wine Reviews and tokenization method

In [ ]:
!pip install kagglehub

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("zynicide/wine-reviews")

print("Path to dataset files:", path)

100%|██████████| 50.9M/50.9M [00:00<00:00, 107MB/s] 

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/zynicide/wine-reviews/versions/4


In [ ]:
import os
files = [file for file in os.listdir(path) if os.path.isfile(os.path.join(path, file))]
print(files)

['winemag-data-130k-v2.csv', 'winemag-data_first150k.csv', 'winemag-data-130k-v2.json']


In [ ]:
import pandas as pd

data = pd.read_csv(f"{path}/winemag-data-130k-v2.csv")

# all categories
#print(data.columns.tolist())

print("Number of rows:", data.shape[0])
print("Missing values:\n", data.isnull().sum())


Number of rows: 129971
Missing values:
 Unnamed: 0                   0
country                     63
description                  0
designation              37465
points                       0
price                     8996
province                    63
region_1                 21247
region_2                 79460
taster_name              26244
taster_twitter_handle    31213
title                        0
variety                      1
winery                       0
dtype: int64


In [ ]:
#remove row with missing values
cleaned_data = data.dropna()
print("Number of rows after cleaning:", cleaned_data.shape[0])


#too less rows after dropping the rows with no data, so decided to fill the missing values insted

Number of rows after cleaning: 22387


In [ ]:
#Filling missing data
data.fillna("", inplace=True)
print("Number of rows:", data.shape[0])

missing_values = data.isnull().sum()
print(missing_values)

<ipython-input-5-f33e01b52957>:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  data.fillna("", inplace=True)


Number of rows: 129971
Unnamed: 0               0
country                  0
description              0
designation              0
points                   0
price                    0
province                 0
region_1                 0
region_2                 0
taster_name              0
taster_twitter_handle    0
title                    0
variety                  0
winery                   0
dtype: int64


In [ ]:
data["processed_text"] = (
    "wine review : " + data["country"].str.strip() + " : " + data["province"].str.strip() + " : " + data["variety"].str.strip() + " : " + data["description"].str.strip()
)


print(data["processed_text"].head())

0    wine review : Italy : Sicily & Sardinia : Whit...
1    wine review : Portugal : Douro : Portuguese Re...
2    wine review : US : Oregon : Pinot Gris : Tart ...
3    wine review : US : Michigan : Riesling : Pinea...
4    wine review : US : Oregon : Pinot Noir : Much ...
Name: processed_text, dtype: object


In [ ]:
!pip install transformers


### Bert - word pair tokenizer

⚠️ Not suitable for text generation, even I dont know why I spent time on this

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Tokenize the processed text
data["tokens"] = data["processed_text"].apply(lambda x: tokenizer(x, padding=True, truncation=True, max_length=512))

# Example: View the first tokenized row
print("First Tokenized Entry:")
print(data["tokens"].iloc[0])

First Tokenized Entry:
{'input_ids': [101, 4511, 3319, 1024, 3304, 1024, 12071, 1004, 21594, 1024, 2317, 12586, 1024, 23958, 2015, 2421, 5133, 5909, 1010, 23528, 1010, 7987, 5714, 9221, 1998, 9550, 12810, 1012, 1996, 14412, 3686, 3475, 1005, 1056, 15241, 22570, 1010, 5378, 4895, 29443, 6675, 6207, 1010, 20418, 1998, 9550, 10878, 4077, 28022, 5648, 3012, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [ ]:
index = 3
processed_text = data["processed_text"].iloc[index]
input_ids = data["tokens"].iloc[index]["input_ids"]
decoded_tokens = [tokenizer.decode([id]) for id in input_ids]
decoded_text = tokenizer.decode(data["tokens"].iloc[index]["input_ids"])


# Print the results
print("Processed Text:")
print(processed_text)
print("\nInput IDs:")
print(input_ids)
print("\nDecoded Tokens:")
print(decoded_tokens)
print("\nDecoded Text:")
print(decoded_text)

Processed Text:
wine review: US : Michigan : Riesling : Pineapple rind, lemon pith and orange blossom start off the aromas. The palate is a bit more opulent, with notes of honey-drizzled guava and mango giving way to a slightly astringent, semidry finish.

Input IDs:
[101, 4511, 3319, 1024, 2149, 1024, 4174, 1024, 15544, 2229, 2989, 1024, 7222, 23804, 15544, 4859, 1010, 14380, 6770, 2232, 1998, 4589, 20593, 2707, 2125, 1996, 23958, 2015, 1012, 1996, 14412, 3686, 2003, 1037, 2978, 2062, 6728, 27581, 1010, 2007, 3964, 1997, 6861, 1011, 2852, 10993, 29247, 2094, 19739, 12462, 1998, 24792, 3228, 2126, 2000, 1037, 3621, 2004, 18886, 25997, 2102, 1010, 4100, 21190, 3926, 1012, 102]

Decoded Tokens:
['[CLS]', 'wine', 'review', ':', 'us', ':', 'michigan', ':', 'ri', '##es', '##ling', ':', 'pine', '##apple', 'ri', '##nd', ',', 'lemon', 'pit', '##h', 'and', 'orange', 'blossom', 'start', 'off', 'the', 'aroma', '##s', '.', 'the', 'pal', '##ate', 'is', 'a', 'bit', 'more', 'op', '##ulent', ',', 'wit

In [ ]:
from collections import Counter

# Flatten all input_ids from your dataset
all_input_ids = [id for tokens in data["tokens"] for id in tokens["input_ids"]]

# Count token occurrences
token_counts = Counter(all_input_ids)

# Map token IDs back to text
id_to_token = {id: tokenizer.decode([id]) for id in token_counts.keys()}

# Create a list of tuples (Token Text, Token ID, Count) and sort by frequency
sorted_tokens_by_frequency = sorted(
    [(id_to_token[token_id], token_id, count) for token_id, count in token_counts.items()],
    key=lambda x: x[2],  # Sort by count (frequency)
    reverse=True         # Descending order
)

print(f"Total Tokens: {total_tokens}")
print(f"Unique Tokens: {len(sorted_tokens_by_frequency)}")
print("\nTop 20 Most Frequent Tokens:")
for token_text, token_id, count in sorted_tokens_by_frequency[:20]:
    print(f"Token: {token_text}, Token ID: {token_id}, Count: {count}")


Total Tokens: 9236768
Unique Tokens: 16620

Top 20 Most Frequent Tokens:
Token: :, Token ID: 1024, Count: 521322
Token: ,, Token ID: 1010, Count: 441931
Token: ., Token ID: 1012, Count: 356241
Token: and, Token ID: 1998, Count: 348172
Token: the, Token ID: 1996, Count: 221530
Token: wine, Token ID: 4511, Count: 212149
Token: a, Token ID: 1037, Count: 180002
Token: of, Token ID: 1997, Count: 173073
Token: review, Token ID: 3319, Count: 129981
Token: [CLS], Token ID: 101, Count: 129971
Token: [SEP], Token ID: 102, Count: 129971
Token: with, Token ID: 2007, Count: 120532
Token: this, Token ID: 2023, Count: 114197
Token: ##s, Token ID: 2015, Count: 97905
Token: is, Token ID: 2003, Count: 96866
Token: -, Token ID: 1011, Count: 89615
Token: it, Token ID: 2009, Count: 86121
Token: ##y, Token ID: 2100, Count: 70794
Token: in, Token ID: 1999, Count: 63212
Token: flavors, Token ID: 26389, Count: 62812


### GPT2 - BPE tokenizer

In [ ]:
%%time

from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

tokenizer.pad_token = tokenizer.eos_token #to fix error with padding


# tested max_length of this dataset, should be fine as now row is longer than 512 hence there wont be any truncation
# learnt padding is important for GPU - but need help here
data["tokens"] = data["processed_text"].apply(
    lambda x: tokenizer(x, padding=True, truncation=True, max_length=512, return_tensors="pt")
)

CPU times: user 1min 24s, sys: 636 ms, total: 1min 25s
Wall time: 1min 35s


In [ ]:
index = 0

processed_text = data["processed_text"].iloc[index]
input_ids = data["tokens"].iloc[index]["input_ids"].squeeze().tolist()  # Convert to a flat list
decoded_tokens = [tokenizer.decode([id]) for id in input_ids]
decoded_text = tokenizer.decode(input_ids)

print("Processed Text:")
print(processed_text)
print("\nInput IDs:")
print(input_ids)
print("\nDecoded Tokens:")
print(decoded_tokens)
print("\nDecoded Text:")
print(decoded_text)


Processed Text:
wine review : Italy : Sicily & Sardinia : White Blend : Aromas include tropical fruit, broom, brimstone and dried herb. The palate isn't overly expressive, offering unripened apple, citrus and dried sage alongside brisk acidity.

Input IDs:
[39002, 2423, 1058, 8031, 1058, 49301, 1222, 46997, 43168, 1058, 2635, 41198, 1058, 317, 398, 292, 2291, 19690, 8234, 11, 47085, 11, 49943, 6440, 290, 16577, 21047, 13, 383, 44100, 2125, 470, 17698, 38084, 11, 6011, 555, 5528, 2945, 17180, 11, 35405, 290, 16577, 35021, 7848, 35984, 7408, 414, 13]

Decoded Tokens:
['wine', ' review', ' :', ' Italy', ' :', ' Sicily', ' &', ' Sard', 'inia', ' :', ' White', ' Blend', ' :', ' A', 'rom', 'as', ' include', ' tropical', ' fruit', ',', ' broom', ',', ' brim', 'stone', ' and', ' dried', ' herb', '.', ' The', ' palate', ' isn', "'t", ' overly', ' expressive', ',', ' offering', ' un', 'rip', 'ened', ' apple', ',', ' citrus', ' and', ' dried', ' sage', ' alongside', ' brisk', ' acid', 'ity', '.']

In [ ]:
#checking if there are rows more than 512 tokens - Result: No rows exceed 512 tokens. (atleast for this dataset)
# don't run this
rows_with_long_tokens = []


for index, row in data.iterrows():
    tokenized = tokenizer(row["processed_text"], truncation=False)
    #tokenized = len(tokenized["input_ids"])

    if len(tokenized["input_ids"]) > 512:
        rows_with_long_tokens.append((index, row["processed_text"], len(tokenized["input_ids"])))

# Display rows with more than 512 tokens
if rows_with_long_tokens:
    long_token_data = pd.DataFrame(rows_with_long_tokens, columns=["Index", "Processed Text", "Token Count"])
else:
    print("No rows exceed 512 tokens.")

print("Maximum Token Count:", max_length)

In [ ]:
max_length = 0
for index, row in data.iterrows():
    split_tokens = data["tokens"].iloc[index]["input_ids"].squeeze().tolist()
    max_length = max(max_length, len(split_tokens))

print("Maximum Token Count:", max_length)

Maximum Token Count: 207


In [ ]:
from collections import Counter

# Flatten all input_ids from the tokenized data
all_input_ids = [
    token_id
    for tokens in data["tokens"]
    for token_id in tokens["input_ids"].squeeze().tolist()
]

# Count token occurrences
token_counts = Counter(all_input_ids)

# Map token IDs back to text
id_to_token = {token_id: tokenizer.decode([token_id]) for token_id in token_counts.keys()}

# Create a list of tuples (Token Text, Token ID, Count) and sort by frequency
sorted_tokens_by_frequency = sorted(
    [(id_to_token[token_id], token_id, count) for token_id, count in token_counts.items()],
    key=lambda x: x[2],  # Sort by count (frequency)
    reverse=True         # Descending order
)

# Convert to DataFrame for visualization
max_rows = 50
top_tokens = pd.DataFrame(sorted_tokens_by_frequency[:max_rows], columns=["Token Text", "Token ID", "Count"])

# Display summary
total_tokens = sum(token_counts.values())
print(f"Total Tokens: {total_tokens}")
print(f"Unique Tokens: {len(sorted_tokens_by_frequency)}")
print(f"\nTop {max_rows} Most Frequent Tokens:")
print(top_tokens)

# limit token to 10k

Total Tokens: 9044969
Unique Tokens: 23051

Top 50 Most Frequent Tokens:
     Token Text  Token ID   Count
0             :      1058  519886
1             ,        11  440537
2             .        13  355484
3           and       290  347182
4            of       286  172870
5           the       262  168174
6             a       257  157732
7          wine     39002  130064
8        review      2423  129979
9          with       351  115651
10           is       318   96695
11            -        12   89417
12         wine      8237   80093
13         this       428   72883
14           to       284   61426
15           in       287   60612
16      flavors     17361   60316
17           US      1294   54506
18          The       383   52648
19           's       338   51481
20        fruit      8234   48682
21          ity       414   44576
22            t       256   44541
23           It       632   43595
24           on       319   43095
25        black      2042   42748
26       

In [ ]:
print(data["tokens"].iloc[0])

{'input_ids': tensor([[39002,  2423,    25,  8031,  1058, 49301,  1222, 46997, 43168,  1058,
          2635, 41198,  1058,   317,   398,   292,  2291, 19690,  8234,    11,
         47085,    11, 49943,  6440,   290, 16577, 21047,    13,   383, 44100,
          2125,   470, 17698, 38084,    11,  6011,   555,  5528,  2945, 17180,
            11, 35405,   290, 16577, 35021,  7848, 35984,  7408,   414,    13]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1]])}


### Saving tokens to excel

In [ ]:
from collections import Counter
import pandas as pd

# Assuming data["tokens"] and tokenizer are defined, reinitialize token counting
all_input_ids = [
    token_id
    for tokens in data["tokens"]
    for token_id in tokens["input_ids"].squeeze().tolist()
]

# Count token occurrences
token_counts = Counter(all_input_ids)

# Map token IDs back to text
id_to_token = {token_id: tokenizer.decode([token_id]) for token_id in token_counts.keys()}

# Create a sorted list of tokens by frequency
sorted_tokens_by_frequency = sorted(
    [(id_to_token[token_id], token_id, count) for token_id, count in token_counts.items()],
    key=lambda x: x[2],  # Sort by count (frequency)
    reverse=True         # Descending order
)

# Create a DataFrame for all tokens
all_tokens_df = pd.DataFrame(sorted_tokens_by_frequency, columns=["Token Text", "Token ID", "Count"])

# Save to Excel
file_path = "all_token_statistics.xlsx"
all_tokens_df.to_excel(file_path, index=False)

print(f"Token statistics for all tokens saved to {file_path}.")



Token statistics for all tokens saved to all_token_statistics.xlsx.


# Making one class

In [ ]:
!pip install kagglehub

In [1]:
import kagglehub
import os
import pandas as pd
from transformers import GPT2Tokenizer
from collections import Counter

In [29]:
class WineReviewProcessor:
    def __init__(self, dataset_name="zynicide/wine-reviews"):
        self.dataset_name = dataset_name
        self.path = kagglehub.dataset_download(self.dataset_name)
        print("Path to dataset files:", self.path)
        self.files = [file for file in os.listdir(self.path) if os.path.isfile(os.path.join(self.path, file))]
        print("Dataset files:", self.files)
        self.data = pd.read_csv(f"{self.path}/winemag-data-130k-v2.csv")
        self.data.fillna("", inplace=True)
        print("Missing values:", self.data.isnull().sum())
        print("Number of rows:", self.data.shape[0])

        self.tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
        self.tokenizer.pad_token = self.tokenizer.eos_token

        self.data["processed_text"] = self._process_text()
        self.data["tokens"] = self._tokenize_text()

    def _process_text(self):
        return (
            "wine review : " +
            self.data["country"].str.strip() + " : " +
            self.data["province"].str.strip() + " : " +
            self.data["variety"].str.strip() + " : " +
            self.data["description"].str.strip()
        )

    def _tokenize_text(self):
        return self.data["processed_text"].apply(
            lambda x: self.tokenizer(
                x,
                padding=True,
                truncation=True,
                max_length=512,
                return_tensors="pt"
            )
        )

    def get_tokenized_data(self, index):
        processed_text = self.data["processed_text"].iloc[index]
        input_ids = self.data["tokens"].iloc[index]["input_ids"].squeeze().tolist()
        decoded_tokens = [self.tokenizer.decode([id]) for id in input_ids]
        decoded_text = self.tokenizer.decode(input_ids)

        return {
            "processed_text": processed_text,
            "input_ids": input_ids,
            "decoded_tokens": decoded_tokens,
            "decoded_text": decoded_text
        }

    def get_max_token_length(self):
        max_length = 0
        for tokens in self.data["tokens"]:
            split_tokens = tokens["input_ids"].squeeze().tolist()
            max_length = max(max_length, len(split_tokens))
        return max_length

    def analyze_tokens(self, max_rows=50):
        all_input_ids = []

    # Collect all token IDs into a flat list
        for tokens in self.data["tokens"]:
            input_ids = tokens["input_ids"]
            if isinstance(input_ids[0], list):  # Handle list of lists
                all_input_ids.extend([token_id for sublist in input_ids for token_id in sublist])
            else:  # Handle flat list
                all_input_ids.extend(input_ids)

    # Count frequencies of token IDs
        token_counts = Counter(all_input_ids)

    # Map token IDs to their decoded texts
        id_to_token = {token_id: self.tokenizer.decode([token_id]) for token_id in token_counts.keys()}
        sorted_tokens_by_frequency = sorted(
            [(id_to_token[token_id], token_id, count) for token_id, count in token_counts.items()],
            key=lambda x: x[2],
            reverse=True,
        )

    # Prepare a DataFrame for the top tokens
        top_tokens = pd.DataFrame(sorted_tokens_by_frequency[:max_rows], columns=["Token Text", "Token ID", "Count"])

        total_tokens = sum(token_counts.values())
        unique_tokens = len(token_counts)

        return {
            "total_tokens": total_tokens,
            "unique_tokens": unique_tokens,
            "top_tokens": top_tokens,
        }

    def limit_tokens(self, max_tokens=10000):
        all_input_ids = [
            token_id
            for tokens in self.data["tokens"]
            for token_id in tokens["input_ids"].squeeze().tolist()
        ]

        token_counts = Counter(all_input_ids)
        sorted_tokens = sorted(token_counts.items(), key=lambda x: x[1], reverse=True)
        top_tokens = {token_id for token_id, _ in sorted_tokens[:max_tokens]}

        def filter_tokens(tokenized_row):
            filtered_input_ids = [
                token_id
                for token_id in tokenized_row["input_ids"].squeeze().tolist()
                if token_id in top_tokens
            ]
            tokenized_row["input_ids"] = [[token_id] for token_id in filtered_input_ids]
            return tokenized_row

        self.data["tokens"] = self.data["tokens"].apply(filter_tokens)

    def get_data_for_training(self):
        return self.data


In [30]:
%%time
processor = WineReviewProcessor()

Path to dataset files: /root/.cache/kagglehub/datasets/zynicide/wine-reviews/versions/4
Dataset files: ['winemag-data-130k-v2.csv', 'winemag-data_first150k.csv', 'winemag-data-130k-v2.json']


<ipython-input-29-071bb989874f>:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  self.data.fillna("", inplace=True)


Missing values: Unnamed: 0               0
country                  0
description              0
designation              0
points                   0
price                    0
province                 0
region_1                 0
region_2                 0
taster_name              0
taster_twitter_handle    0
title                    0
variety                  0
winery                   0
dtype: int64
Number of rows: 129971
CPU times: user 1min 56s, sys: 558 ms, total: 1min 57s
Wall time: 2min 3s


In [35]:
# Get tokenized data for a specific index
data_summary = processor.get_tokenized_data(index=0)
print("Processed Text:", data_summary["processed_text"])
print("Input IDs:", data_summary["input_ids"])
print("Decoded Tokens:", data_summary["decoded_tokens"])
print("Decoded Text:", data_summary["decoded_text"])

AttributeError: 'list' object has no attribute 'squeeze'

In [25]:
# Maximum token length
max_token_length = processor.get_max_token_length()
print("Maximum Token Count:", max_token_length)

Maximum Token Count: 207


In [32]:
# Analyze tokens
token_analysis = processor.analyze_tokens()
print(f"Total Tokens: {token_analysis['total_tokens']}")
print(f"Unique Tokens: {token_analysis['unique_tokens']}")
#print(f"Top Tokens:\n{token_analysis['top_tokens']}")

TypeError: int() argument must be a string, a bytes-like object or a real number, not 'list'

In [33]:
processor.limit_tokens()

In [ ]:
# Analyze tokens
token_analysis = processor.analyze_tokens()
print(f"Total Tokens: {token_analysis['total_tokens']}")
print(f"Unique Tokens: {token_analysis['unique_tokens']}")

In [34]:
processor.get_data_for_training()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery,processed_text,tokens
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,,Sicily & Sardinia,Etna,,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia,wine review : Italy : Sicily & Sardinia : Whit...,"[input_ids, attention_mask]"
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,,,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,wine review : Portugal : Douro : Portuguese Re...,"[input_ids, attention_mask]"
2,2,US,"Tart and snappy, the flavors of lime flesh and...",,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,wine review : US : Oregon : Pinot Gris : Tart ...,"[input_ids, attention_mask]"
3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,,Alexander Peartree,,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,wine review : US : Michigan : Riesling : Pinea...,"[input_ids, attention_mask]"
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,wine review : US : Oregon : Pinot Noir : Much ...,"[input_ids, attention_mask]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129966,129966,Germany,Notes of honeysuckle and cantaloupe sweeten th...,Brauneberger Juffer-Sonnenuhr Spätlese,90,28.0,Mosel,,,Anna Lee C. Iijima,,Dr. H. Thanisch (Erben Müller-Burggraef) 2013 ...,Riesling,Dr. H. Thanisch (Erben Müller-Burggraef),wine review : Germany : Mosel : Riesling : Not...,"[input_ids, attention_mask]"
129967,129967,US,Citation is given as much as a decade of bottl...,,90,75.0,Oregon,Oregon,Oregon Other,Paul Gregutt,@paulgwine,Citation 2004 Pinot Noir (Oregon),Pinot Noir,Citation,wine review : US : Oregon : Pinot Noir : Citat...,"[input_ids, attention_mask]"
129968,129968,France,Well-drained gravel soil gives this wine its c...,Kritt,90,30.0,Alsace,Alsace,,Roger Voss,@vossroger,Domaine Gresser 2013 Kritt Gewurztraminer (Als...,Gewürztraminer,Domaine Gresser,wine review : France : Alsace : Gewürztraminer...,"[input_ids, attention_mask]"
129969,129969,France,"A dry style of Pinot Gris, this is crisp with ...",,90,32.0,Alsace,Alsace,,Roger Voss,@vossroger,Domaine Marcel Deiss 2012 Pinot Gris (Alsace),Pinot Gris,Domaine Marcel Deiss,wine review : France : Alsace : Pinot Gris : A...,"[input_ids, attention_mask]"


# GPT

In [46]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset, DataLoader

class GPTDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        input_ids = torch.tensor(self.data.iloc[idx]["tokens"]["input_ids"], dtype=torch.long)
        return {"input_ids": input_ids}

class GPT2Block(nn.Module):
    def __init__(self, embed_dim, num_heads, ff_dim, dropout=0.1):
        super().__init__()
        self.attention = nn.MultiheadAttention(embed_dim, num_heads, dropout=dropout)
        self.norm1 = nn.LayerNorm(embed_dim)
        self.norm2 = nn.LayerNorm(embed_dim)
        self.ffn = nn.Sequential(
            nn.Linear(embed_dim, ff_dim),
            nn.ReLU(),
            nn.Linear(ff_dim, embed_dim)
        )
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, attn_mask=None):
        attn_output, _ = self.attention(x, x, x, attn_mask=attn_mask, need_weights=False)
        x = self.norm1(x + self.dropout(attn_output))
        ffn_output = self.ffn(x)
        x = self.norm2(x + self.dropout(ffn_output))
        return x

class CustomGPT(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_heads, num_layers, ff_dim, max_len, dropout=0.1):
        super().__init__()
        self.token_embedding = nn.Embedding(vocab_size, embed_dim)
        self.position_embedding = nn.Embedding(max_len, embed_dim)
        self.layers = nn.ModuleList([
            GPT2Block(embed_dim, num_heads, ff_dim, dropout) for _ in range(num_layers)
        ])
        self.ln_f = nn.LayerNorm(embed_dim)
        self.head = nn.Linear(embed_dim, vocab_size, bias=False)
        self.max_len = max_len

    def forward(self, input_ids):
        # Input shape: (batch_size, seq_len)
        batch_size, seq_len = input_ids.size()

        # Create position ids and embeddings
        positions = torch.arange(seq_len, device=input_ids.device).unsqueeze(0).expand(batch_size, seq_len)
        token_embeddings = self.token_embedding(input_ids)  # (batch_size, seq_len, embed_dim)
        position_embeddings = self.position_embedding(positions)  # (batch_size, seq_len, embed_dim)

        # Combine token and position embeddings
        x = token_embeddings + position_embeddings

        # Transpose for MultiheadAttention: (seq_len, batch_size, embed_dim)
        x = x.transpose(0, 1)

        for layer in self.layers:
            x = layer(x)

        # Transpose back to (batch_size, seq_len, embed_dim)
        x = x.transpose(0, 1)

        x = self.ln_f(x)  # Layer norm
        logits = self.head(x)  # Output logits
        return logits

# Create the GPT class
class GPTTrainer:
    def __init__(self, vocab_size, embed_dim=256, num_heads=8, num_layers=8, ff_dim=1024, max_len=512, dropout=0.1):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model = CustomGPT(vocab_size, embed_dim, num_heads, num_layers, ff_dim, max_len, dropout).to(self.device)
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = torch.optim.AdamW(self.model.parameters(), lr=5e-4)

    def train(self, train_loader, epochs=3):
        self.model.train()
        for epoch in range(epochs):
            epoch_loss = 0
            for batch in train_loader:
                input_ids = batch["input_ids"].to(self.device)
                labels = input_ids.clone()

                self.optimizer.zero_grad()
                logits = self.model(input_ids)
                loss = self.criterion(logits.view(-1, logits.size(-1)), labels.view(-1))
                loss.backward()
                self.optimizer.step()

                epoch_loss += loss.item()

            print(f"Epoch {epoch + 1}/{epochs}, Loss: {epoch_loss / len(train_loader)}")

    def save_model(self, path):
        torch.save(self.model.state_dict(), path)

def collate_fn(batch):
    input_ids = [item["input_ids"].squeeze(1) if len(item["input_ids"].shape) > 1 else item["input_ids"] for item in batch]
    padded_input_ids = pad_sequence(input_ids, batch_first=True, padding_value=0)  # 0 is the padding token ID
    return {"input_ids": padded_input_ids}

In [ ]:
train_data = processor.get_data_for_training()
dataset = GPTDataset(train_data)
train_loader = DataLoader(dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)
gpt_trainer = GPTTrainer(vocab_size=len(processor.tokenizer))
gpt_trainer.train(train_loader, epochs=5)
gpt_trainer.save_model("custom_gpt_model.pth")